In [61]:
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [62]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack, csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import Word2Vec

In [63]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/anoop-vs/nlp-climate-change/refs/heads/main/main_data.csv"
)

In [64]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /Users/nafis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nafis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/nafis/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [65]:
def preprocess(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = text.split()
    stopword_list = set(stopwords.words("english"))
    words = [w for w in words if w not in stopword_list]
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    text = " ".join(words)
    return text

In [66]:
df["content"] = df["content"].apply(preprocess)

In [67]:
X = df["content"]  # 1D Series of text
y = df["label"]

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# Oversample BEFORE splitting (OG behavior)
oversampler = RandomOverSampler(
    sampling_strategy={"Negative": 1842, "Positive": 1822, "Neutral": 1842}
)
X_resampled, y_resampled = oversampler.fit_resample(X.to_frame(name="content"), y)

# Rebuild DataFrame and reset X, y to the resampled data
df_resampled = pd.concat([X_resampled["content"], y_resampled], axis=1)
X = df_resampled["content"]
y = df_resampled["label"]

# Train Word2Vec on the full resampled corpus (OG-style)
X_corpus_tokens = X.apply(str.split).tolist()
w2v_model = Word2Vec(X_corpus_tokens, vector_size=100, window=5, min_count=1, workers=4)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### COUNTVEC-WORD2VEC - LOGISTIC REGRESSION


In [ ]:
count_vectorizer = CountVectorizer()

In [78]:
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [79]:
# Prepare token lists like the OG approach
X_train_tokens = X_train.apply(str.split).tolist()
X_test_tokens = X_test.apply(str.split).tolist()

# Reuse the Word2Vec model trained on the full (resampled) corpus
# Average per-document embeddings (OG style)
X_train_w2v = np.array(
    [
        np.mean([w2v_model.wv[w] for w in document], axis=0)
        for document in X_train_tokens
    ]
)
X_test_w2v = np.array(
    [np.mean([w2v_model.wv[w] for w in document], axis=0) for document in X_test_tokens]
)

In [80]:
# Combine sparse CountVectorizer features with dense W2V (ndarray)
print("Shapes:")
print("X_train_count:", X_train_count.shape, " X_train_w2v:", X_train_w2v.shape)
print("X_test_count:", X_test_count.shape, " X_test_w2v:", X_test_w2v.shape)

X_train_combined = hstack([X_train_count, X_train_w2v])
X_test_combined = hstack([X_test_count, X_test_w2v])

Shapes:
X_train_count: (4404, 5000)  X_train_w2v: (4404, 100)
X_test_count: (1102, 5000)  X_test_w2v: (1102, 100)


In [81]:
lrcw = LogisticRegression()
lrcw.fit(X_train_combined, y_train)

/Users/nafis/code/python/climate change sentiment analysis/.venv311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [82]:
y_pred = lrcw.predict(X_test_combined)

In [83]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.89      0.96      0.92       389
     Neutral       0.80      0.70      0.75       370
    Positive       0.76      0.78      0.77       343

    accuracy                           0.82      1102
   macro avg       0.81      0.81      0.81      1102
weighted avg       0.82      0.82      0.81      1102

